In [12]:
import pandas as pd

In [13]:
import numpy as np

In [14]:
import os

In [16]:
df = pd.read_excel(r".\FSIS-Recall-Summary-2014.xlsx", header=1, index =2)


In [17]:
df.head(5)

,Recall Date,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
0,2014-01-10,001-2014,I,Mechanically Separated Chicken Products,Salmonella,33840
1,2014-01-13,002-2014,I,Various Beef Products,Other,42103
2,2014-01-15,003-2014,I,Beef Franks,Undeclared Allergen,2664
3,2014-01-17,004-2014,II,Beef and Pork Products,Undeclared Allergen,130000
4,2014-01-17,005-2014,I,Spiral Hams,Listeria monocytogenes,67113


In [18]:
pd.DataFrame(df)
           

,Recall Date,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
0,2014-01-10,001-2014,I,Mechanically Separated Chicken Products,Salmonella,33840
1,2014-01-13,002-2014,I,Various Beef Products,Other,42103
2,2014-01-15,003-2014,I,Beef Franks,Undeclared Allergen,2664
3,2014-01-17,004-2014,II,Beef and Pork Products,Undeclared Allergen,130000
4,2014-01-17,005-2014,I,Spiral Hams,Listeria monocytogenes,67113
...,...,...,...,...,...,...
89,2014-12-27,090-2014,I,Pork products,Undeclared Allergen,83666
90,2014-12-27,091-2014,I,Chicken and Beef Products,Undeclared Allergen,55813
91,2014-12-29,092-2014,I,Beef and Pork Products,Undeclared Allergen,38400
92,2014-12-31,093-2014,I,Chicken Products,Undeclared Allergen,5300
